In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# loading the dataframe
df_main = pd.read_csv('sample30.csv')

In [4]:
df_reco = df_main[['name','reviews_username','reviews_rating']]

In [5]:
# check for null values

df_reco.isnull().sum()

name                 0
reviews_username    63
reviews_rating       0
dtype: int64

In [6]:
# dropping rows with NaN values

df_reco.dropna(axis=0, inplace=True)

In [7]:
# check for duplicates

df_reco.duplicated().sum()

2198

In [8]:
# dropping the duplicates

df_reco.drop_duplicates(inplace=True)

In [9]:
# check for duplicates based on subset of 'name' and 'reviews_username'

df_reco.duplicated(subset=['name','reviews_username']).sum()

151

From above we note that we have certain users who have given two or more different ratings to the same products. Let us impute those ratings the average ratings.

In [10]:
# get the average ratings for duplicated rows

df_mean = df_reco[df_reco.duplicated(subset=['name','reviews_username'], keep=False)].groupby(
    by=['name', 'reviews_username']).mean()

df_mean

reviews_rating
name                                               reviews_username                 
100:Complete First Season (blu-Ray)                dontdodat                3.500000
Alex Cross (dvdvideo)                              mookie                   4.500000
Aveeno Baby Continuous Protection Lotion Sunscr... byamazon customer        2.666667
Avery174 Ready Index Contemporary Table Of Cont... gellis                   4.500000
                                                   the office guro          3.500000
...                                                                              ...
Tostitos Bite Size Tortilla Chips                  debb                     4.000000
                                                   rick                     4.500000
                                                   sandy                    3.000000
Windex Original Glass Cleaner Refill 67.6oz (2 ... laura                    1.500000
                                                   suri                     3.500000

[135 rows x 1 columns]

In [11]:
# left merge the df_reco and df_mean dataframes over 'name' and 'reviews_username'

df_merged = df_reco.merge(df_mean, how='left' , on=['name', 'reviews_username']).sort_values(
    by=['name', 'reviews_username'])

df_merged.head()

,name,reviews_username,reviews_rating_x,reviews_rating_y
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3,NaN
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5,NaN
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5,NaN
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3,NaN
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5,NaN


In [12]:
df_merged.isnull().sum()

name                    0
reviews_username        0
reviews_rating_x        0
reviews_rating_y    27453
dtype: int64

Since we have very few such rows, we would have many NaN values in reviews_rating_y column. Let us concatenate the dataframe such that we get average values from reviews_rating_y column and normal ratings from reviews_rating_x column

In [13]:
A = df_merged[df_merged['reviews_rating_y'].isnull()][['name','reviews_username','reviews_rating_x']]
A.rename(columns={'reviews_rating_x':'reviews_rating_final'}, inplace=True)

B = df_merged[df_merged['reviews_rating_y'].isnull()==False][['name','reviews_username','reviews_rating_y']]
B.rename(columns={'reviews_rating_y':'reviews_rating_final'}, inplace=True)

df_final= pd.concat([A, B])
df_final.sort_values(by=['name', 'reviews_username'])

,name,reviews_username,reviews_rating_final
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3.0
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5.0
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5.0
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3.0
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5.0
...,...,...,...
20525,Yes To Grapefruit Rejuvenating Body Wash,sheila,1.0
20534,Yes To Grapefruit Rejuvenating Body Wash,skeel,4.0
20531,Yes To Grapefruit Rejuvenating Body Wash,td33,3.0
20553,Yes To Grapefruit Rejuvenating Body Wash,trishaxo2u,5.0


In [14]:
# check for duplicates from `df_final` df
df_final.duplicated().sum()

151

In [15]:
# drop duplicates from 'df_final' df

df_final.drop_duplicates(inplace=True)

In [16]:
df_final.head()

,name,reviews_username,reviews_rating_final
19543,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,brewno,3.0
19545,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,embum,5.0
19547,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,granny,5.0
19542,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,smokey bear,3.0
19546,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,spicesea,5.0


In [17]:
df_final['reviews_rating_final'].value_counts()

5.000000    19456
4.000000     5434
3.000000     1183
1.000000     1048
2.000000      369
4.500000       75
3.500000       15
2.500000        2
1.500000        2
3.333333        2
2.666667        1
3.666667        1
Name: reviews_rating_final, dtype: int64

    R: rating matrix (user X item)
    P: User features matrix (user X latent feature)
    Q: Item features matrix (item X laten feature)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter

In [18]:
# Create a user-product matrix.
df_pivot = df_final.pivot(
    index='reviews_username',
    columns='name',
    values='reviews_rating_final'
).fillna(0)


R = np.array(df_pivot)
# N: num of User
N = R.shape[0]
# M: num of Movie
M = R.shape[1]
# K: latent features
K = 50

P = np.random.rand(N,K)
Q = np.random.rand(M,K)


In [19]:
def matrix_factorization(R, P, Q, K, steps=50, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

#         eR = np.dot(P,Q)

        # checking the error
        e = 0
        for i in range(len(R)):

            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))   # applying L2 regularisation
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [20]:
def matrix_factorization_optimised(R, P, Q, K, steps=50, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    
                    # calculate gradient with a and beta parameter
                    p_update = P[i,:] + alpha * (2 * eij * Q[:,j].T - beta * P[i,:])
                    q_update = Q[:,j] + alpha * (2 * eij * P[i,:].T - beta * Q[:,j])
                    #update the values
                    P[i,:] = p_update
                    Q[:,j] = q_update

#         eR = np.dot(P,Q)

        # checking the error
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))   # applying L2 regularisation
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [21]:
nP, nQ = matrix_factorization_optimised(R, P, Q, K)

nR = np.dot(nP, nQ.T)

In [22]:
pred_R = pd.DataFrame(nR, columns = df_pivot.columns, index = df_pivot.index)

In [23]:
# checking predicted ratings with MF method
pred_R

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,6.801422,5.459206,9.694991,6.346352,10.913477,8.527729,12.049012,7.374431,9.756364,4.323451,...,8.799984,11.524764,4.487911,5.215784,11.909475,6.266718,10.433767,4.665540,3.813949,4.828406
00sab00,6.698338,5.180330,8.696871,6.846665,10.253938,6.776632,10.930480,7.440846,9.136365,4.644223,...,8.713510,10.650781,5.055772,5.522302,10.185460,4.470471,9.481095,2.994390,4.301718,3.418069
01impala,6.624298,4.447966,7.535481,6.852279,9.960607,6.239057,12.208640,6.735039,8.337044,4.302333,...,8.813025,10.824174,4.573290,4.008754,10.362675,7.025489,10.056825,3.805366,4.713948,3.591549
02dakota,6.237145,4.397788,8.221986,7.671956,9.533946,7.138449,11.218281,7.018485,9.530731,4.395417,...,9.186681,10.397768,4.413546,5.373433,11.230556,5.860527,10.345128,3.845879,5.163056,4.087821
02deuce,7.026537,4.418772,10.341251,7.820994,10.738306,7.623962,11.981317,6.927422,9.375302,4.688674,...,8.093355,11.746779,4.176657,5.126826,11.044528,5.593675,11.182444,3.760489,4.731085,4.937697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zxcsdfd,5.731920,4.717393,8.007003,7.173130,11.172376,6.918878,11.242848,6.348428,8.980303,5.166262,...,9.210653,11.630585,4.720515,5.353818,10.701398,4.884386,11.803395,4.108929,5.268506,3.611626
zxjki,6.310286,4.665007,8.038766,5.546347,11.283904,6.351929,12.051584,6.226666,9.108081,4.328833,...,9.754447,10.247317,4.302531,5.220735,10.933526,6.282648,11.492700,4.480780,4.250731,4.594359
zyiah4,6.543503,4.759502,7.253734,6.251316,10.044546,6.762739,10.527848,7.318046,9.459199,4.169180,...,8.246224,9.908159,4.093996,4.835763,9.749325,4.649733,9.215091,3.570867,4.080637,3.132022


------------------------------------------------------------------------------------------------------------